In [1]:
import glob
import csv
import pandas
import os
import re
import time
import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
contain_string = config['DEFAULT']['Contain-String']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [2]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [3]:
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 14


In [4]:
dic_collection = {}
for i in collections_twitter:
    #if i.startswith("20") and "Australia" in i:
    if i.startswith("20") and contain_string in i:
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20471
2018_W51_Twitter_Other: 8978243
2018_W52_Twitter_Australia: 38065
2018_W52_Twitter_Other: 16940708
2019_W01_Twitter_Australia: 40880
2019_W01_Twitter_Other: 16994959
2019_W02_Twitter_Australia: 37645
2019_W02_Twitter_Other: 14474627
2019_W03_Twitter_Australia: 52348
2019_W03_Twitter_Other: 20416992
2019_W04_Twitter_Australia: 59625
2019_W04_Twitter_Other: 23417927
2019_W05_Twitter_Australia: 61617
2019_W05_Twitter_Other: 23917345
2019_W06_Twitter_Australia: 61435
2019_W06_Twitter_Other: 24176122
2019_W07_Twitter_Australia: 28017
2019_W07_Twitter_Other: 10484648
2019_W08_Twitter_Australia: 60035
2019_W08_Twitter_Other: 23446630
2019_W09_Twitter_Australia: 58316
2019_W09_Twitter_Other: 22293795
2019_W10_Twitter_Australia: 40767
2019_W10_Twitter_Other: 15941350
2019_W11_Twitter_Australia: 68593
2019_W11_Twitter_Other: 25345811
2019_W12_Twitter_Australia: 50447
2019_W12_Twitter_Other: 18090490
2019_W13_Twitter_Australia: 62926
2019_W13_Twitter_Other: 24379

# Produce links between #tags

In [5]:
pipeline = [
    {"$match": { "entities.hashtags": {"$exists":True,"$ne":[]}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": {
            "hashtags": "$entities.hashtags"
        },
        "count": { "$sum": 1 },
        }
    }
]

In [6]:
# create foler if not exist
def create_folder():
    folder = "output/links_hashtags/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder

In [7]:
# delete existed collection from the list dic_collection
def delete_collection(folder,dic_collection):
    for input_file in glob.glob(os.path.join(folder,'*.csv')):
        collection_name = re.search('{(.+?)}', input_file).group(1)
        if collection_name in dic_collection:
            print("Existed collection: " + collection_name)
            del dic_collection[collection_name]
    return dic_collection

In [8]:
# append to dictionary
def append_dic(data_format,exist,dic):
    data_format.append(dic)
    exist = 1
    return data_format,exist

In [9]:
# sort hashtag
def sort_hashtag(text1, text2):
    l = [text1,text2]
    sort_l = sorted(l)
    return sort_l

In [10]:
#method for the loop
def floop(length,hashtags,data_format,text1):
    for j in range(1,length):
        exist = 0
        # get hashtag2
        text2 = hashtags[j]["text"].lower()
        if(re.match("^[a-zA-Z0-9]*$",text2)):
            sort_l = sort_hashtag(text1, text2)
            text1 = sort_l[0]
            text2 = sort_l[1]
            dic = {"hashtags": text1+ "," + text2, "count" : count}

            if len(data_format) > 0 :
                for d in data_format:
                    if (text1 in d["hashtags"]) and (text2 in d["hashtags"]):
                        d["count"] += count
                        exist = 1
                if exist == 0:
                    data_format,exist = append_dic(data_format,exist,dic)
            else:        
                data_format,exist = append_dic(data_format,exist,dic)
    return data_format, exist

In [11]:
# write csv file
def write_csv(collection,data_format):
    
    file_name = "output/links_hashtags/{" + collection + "}_links_hashtags.csv"
    with open(file_name, 'w') as f:
        # header
        f.write('hashtag,linked_hashtag,number\n')

        for data in data_format:
            hashtag1 = data['hashtags'].split(",")[0]
            hashtag2 = data['hashtags'].split(",")[1]

            line = hashtag1 + ',' + hashtag2 + ',' + str(data["count"]) + '\n'
            f.write(line)

In [ ]:
#create folder if not exist
folder = create_folder()
    
# delete existed collection from the list dic_collection
dic_collection = delete_collection(folder,dic_collection)
       
for collection in sorted(dic_collection):
    print("-----------------------\n")
    print("Processing on collection: " + collection)
    
    data_format = []
    data_list = list(db_twitter[collection].aggregate(pipeline,allowDiskUse=True))

    if len(data_list) > 0:
        for data in data_list:
            hashtags = data["_id"]["hashtags"]
            count = data["count"]
            length = len(hashtags)
            if length > 1 :
                for i in range(0,length-1):
                    # get hashtag1
                    text1 = hashtags[i]["text"].lower()
                    # check if it is in English
                    if(re.match("^[a-zA-Z0-9]*$",text1)):
                        data_format, exist = floop(length,hashtags,data_format,text1)
            else: pass

    print("linked hashtag list is finished")   
    
    write_csv(collection,data_format)

    print ("linked hashtag csv file for collection " + collection + ' is finished.')
    print("-----------------------\n")